In [2]:
import librosa
import soundfile as sf
import numpy as np
import os
import random

In [3]:
COMPRESSION_FACTOR = 0.4
LIST_OF_WORDS = []  # Read from .csv file fra Johan

In [ ]:
def reverse_audio(in_path, word, feeling):
    """Reverse the audio"""
    y, sr = librosa.load(in_path, sr=None)
    rev = y[::-1]  # Reverse the audio array
    out_path = f"sound_files/reverse/{feeling}/{word}_reversed.wav"
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    sf.write(out_path, rev, sr)
    return out_path

# for file in os.listdir("sound_files/original_sound/neutrale-ord-real"):
#     if file.endswith(".wav"):
#         word = file[:-4]  # Remove .wav extension
#         in_path = f"sound_files/original_sound/neutrale-ord-real/{file}"
#         out_path_reversed = reverse_audio(in_path, word, "neutrale")

In [ ]:
def time_compress(in_path, word, feeling, factor=COMPRESSION_FACTOR):
    """Time-compress by factor"""
    y, sr = librosa.load(in_path, sr=None)
    y_fast = librosa.effects.time_stretch(y, rate=1 + factor)
    out_path = f"sound_files/primes/{feeling}/{word}_compressed.wav"
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    sf.write(out_path, y_fast, sr)
    return out_path

# for file in os.listdir("sound_files/original_sound/negative-ord-real"):
#     if file.endswith(".wav"):
#         word = file[:-4]  # Remove .wav extension
#         in_path = f"sound_files/original_sound/negative-ord-real/{file}"
#         out_path_compressed = time_compress(in_path, word, "negative")

In [ ]:
y, sr = librosa.load("sound_files/original_sound/positive-ord-real/blomstre.wav", sr=None)
y_fast = librosa.effects.time_stretch(y, rate=3.0)
y_fast

AttributeError: 'numpy.ndarray' object has no attribute 'play'

In [ ]:
from IPython.display import Audio, display

# Play original and time-stretched audio inline (no files written)
display(Audio(y, rate=sr))
display(Audio(y_fast, rate=sr))

In [22]:
def create_silence(duration_ms, sr=22050):
    """Create silence array"""
    duration_sec = duration_ms / 1000.0
    samples = int(duration_sec * sr)
    return np.zeros(samples)

def concatenate_audio(audio_files, silence_duration_ms=100):
    """Concatenate audio files with silence between them"""
    sr = None
    audio_segments = []
    
    for audio_file in audio_files:
        y, current_sr = librosa.load(audio_file, sr=sr)
        if sr is None:
            sr = current_sr
        audio_segments.append(y)
        if audio_file != audio_files[-1]:  # Add silence except after last file
            audio_segments.append(create_silence(silence_duration_ms, sr))
    
    combined = np.concatenate(audio_segments)
    return combined, sr

In [24]:
def create_trial(word):
    """Create one trial: random masks from other primes"""
    prime = f"primes/{word}_compressed.wav"
    others = [w for w in os.listdir("reverse") if w != f"{word}_reversed.wav"]
    forward_mask_word = random.choice(others)
    backward_mask_word = random.choice([w for w in others if w != forward_mask_word])

    audio_files = [forward_mask_word, prime, backward_mask_word]
    combined, sr = concatenate_audio(audio_files, silence_duration_ms=100)

    out_path = os.path.join("trials", f"{word}_trial.wav")
    sf.write(out_path, combined, sr)
    return out_path, forward_mask_word, backward_mask_word

for word in LIST_OF_WORDS:
    trial, fwd, bwd = create_trial(word)
    with open("trials_log.txt", "a") as log_file:
        log_file.write(f"{word},{fwd},{bwd}\n")
    print(f"Trial for '{word}' created → {trial}")